<a href="https://colab.research.google.com/github/prgmr99/deep_learning-pytorch-/blob/main/LR_nn_Module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **nn.Module로 구현하는 선형 회귀**

이전까지는 선형 회귀를 좀 더 직접적으로 이해하기 위해 가설, 비용 함수를 직접 정의해서 선형 회귀 모델을 구현했다.

이번에는 파이토치에서 이미 구현되어져 제공되고 있는 함수들을 불러오는 것으로 더 쉽게 선형 회귀 모델을 구현해보겠다.


In [4]:
# 파이토치에서는 선형 회귀 모델이 nn.Linear()라는 함수로,
# 평균 제곱오차가 nn.function.mse_loss()라는 함수로 구현되어져 있다.

# 사용 예제
import torch.nn as nn
#model=nn.Linear(input_dim, output_dim) # 에러 떠서 주석처리

import torch.nn.functional as F
#cost=F.mse_loss(prediction, y_train)

### **단순 선형 회귀 구현하기**

In [5]:
# 필요한 도구 임포트
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1)

In [6]:
# 데이터 선언(y=2*x를 가정한 상태에서 데이터 만듬)
# w=2, b=0임을 알고 있기 때문에 w,b 값을 제대로 찾아내도록 하는 것이 목표.
x_train=torch.FloatTensor(([1],[2],[3]))
y_train=torch.FloatTensor(([2],[4],[6]))

In [8]:
# 선형 회귀 모델 구현
# nn.Linear()는 입력의 차원, 출력의 차원을 인수로 받는다.

# 모델을 선언/초기화. 단순 선형 회귀이므로 input_dim=1, output_dim=1
model=nn.Linear(1,1)

# 입력 차원과 출력 차원 모두 1을 인수로 사용.
# model에는 가중치 w와 편향 b가 저장되어져 있다.
# 이 값들은 model.parameters()라는 함수를 사용해 불러올 수 있다.
print(list(model.parameters()))

# 두 값 모두 현재는 랜덤 초기화가 되어져 있다. 
# 두 값 모두 학습의 대상이므로 requires_grad=True가 되어져 있다.

[Parameter containing:
tensor([[-0.1939]], requires_grad=True), Parameter containing:
tensor([0.4694], requires_grad=True)]


In [9]:
# 옵티마이저 정의
# model.parameters()를 사용하여 w와 b를 전달

# optimizer 설정. 경사 하강법 SGD 사용, lr=0.01
optimizer=torch.optim.SGD(model.parameters(),lr=0.01)

In [11]:
nb_epochs=2000
for epoch in range(nb_epochs+1):

  # H(x) 계산
  prediction=model(x_train)

  # cost 계산
  cost=F.mse_loss(prediction,y_train) # 파이토치에서 제공하는 평균 제곱 오차 함수

  # cost로 H(x) 개선하는 부분
  # gradient를 0으로 초기화
  optimizer.zero_grad()

  # 비용 함수를 미분하여 gradient 계산
  cost.backward()

  # W와 b를 업데이트
  optimizer.step()

  if epoch%100==0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch,nb_epochs,cost.item()
    ))

Epoch    0/2000 Cost: 18.562185
Epoch  100/2000 Cost: 0.128051
Epoch  200/2000 Cost: 0.079128
Epoch  300/2000 Cost: 0.048896
Epoch  400/2000 Cost: 0.030215
Epoch  500/2000 Cost: 0.018671
Epoch  600/2000 Cost: 0.011538
Epoch  700/2000 Cost: 0.007129
Epoch  800/2000 Cost: 0.004406
Epoch  900/2000 Cost: 0.002722
Epoch 1000/2000 Cost: 0.001682
Epoch 1100/2000 Cost: 0.001040
Epoch 1200/2000 Cost: 0.000642
Epoch 1300/2000 Cost: 0.000397
Epoch 1400/2000 Cost: 0.000245
Epoch 1500/2000 Cost: 0.000152
Epoch 1600/2000 Cost: 0.000094
Epoch 1700/2000 Cost: 0.000058
Epoch 1800/2000 Cost: 0.000036
Epoch 1900/2000 Cost: 0.000022
Epoch 2000/2000 Cost: 0.000014


In [12]:
# 임의의 입력 4를 선언
new_var=torch.FloatTensor([[4.0]])

# 입력한 값 4에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y=model(new_var) # forward 연산**?
# y=2x -> 입력=4라면, y=8에 가까운 값이 나와야 제대로 학습된 것.

print("훈련 후 입력이 4일 때의 예측값: ", pred_y)

훈련 후 입력이 4일 때의 예측값:  tensor([[7.9926]], grad_fn=<AddmmBackward0>)


In [13]:
# 학습 후의 w와 b의 값 출력
print(list(model.parameters()))

[Parameter containing:
tensor([[1.9957]], requires_grad=True), Parameter containing:
tensor([0.0097], requires_grad=True)]




*   H(x) 식에 입력 x로부터 예측된 y를 얻는 것을 forward 연산이라고 한다.
*   학습 전, prediction=model(x_train)은 x_train으로부터 예측값을 리턴하므로 forward연산
*   학습 후, pred_y=model(new_var)는 임의의 값 new_var로부터 예측값을 리턴하므로 forward연산
*   학습 과정에서 비용함수를 미분하여 기울기를 구하는 것 -> backward 연산
*   cost.backward()는 비용 함수로부터 기울기를 구하라는 의미 -> backward 연산



